In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import mne
from functions_InverseSolutions import showResult, prepareInverseSolution_group, getMNIcoordinates

tmin, tmax = -0.1, 0.6

source_MainFolder = "..\SourceLocalization\SourceEstimates\Data="
sourceFolder = source_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Source folder: ', sourceFolder)

statResultsFolder  = '..\SourceLocalization\Results\Data='
statResultsFolder = statResultsFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Statisctics results folder: ', statResultsFolder)    


subjects_dir = '..\SourceLocalization\subjects\\'
print('Subjects directory: ', subjects_dir)

tstep= 0.01
inv_sol_method = 'beamformer'
smoothAmount=70

In [ ]:
# Load stc for time range of interest [0.1, 0.4]
tminData_cls_tmp, tmaxData_cls_tmp = 0.1, 0.4

stc_real_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData_cls_tmp)+'_'+str(tmaxData_cls_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(statResultsFolder+stc_real_filename)
stc_fsave_all_real_clfRange = np.load(statResultsFolder+stc_real_filename)
print('shape of real sounds source estimates: ', stc_fsave_all_real_clfRange.shape)

# Load stc for baseline [-0.04, 0]
tminData_baseline_tmp, tmaxData_baseline_tmp = -0.04, 0

stc_real_filename_baseline = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(stc_real_filename_baseline)
stc_fsave_all_real_baseline = np.load(statResultsFolder+stc_real_filename_baseline)
print('shape of real sounds baseline source estimates: ', stc_fsave_all_real_baseline.shape)

# take mean over participants (dimension 2)
stc_fsave_all_real_clfRange_avg = np.mean(stc_fsave_all_real_clfRange, axis=2)
print('Shape stc_fsave_all_real_clfRange_avg after avg over subjects: ', stc_fsave_all_real_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_clfRange_avg= np.mean(stc_fsave_all_real_clfRange_avg, axis=1)
print('Shape stc_fsave_all_real_clfRange_avg after avg across time: ', stc_fsave_all_real_clfRange_avg.shape)
stc_fsave_all_real_clfRange_avg.shape

# take mean over participants (dimension 2)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline, axis=2)
print('Shape stc_fsave_all_real_baseline_avg after avg over subjects: ', stc_fsave_all_real_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline_avg, axis=1)
print('Shape stc_fsave_all_real_baseline_avg after avg across time: ', stc_fsave_all_real_baseline_avg.shape)
stc_fsave_all_real_baseline_avg.shape

# Take difference post-base normalized:
stc_fsave_all_real_diff_avg = 100*(stc_fsave_all_real_clfRange_avg - stc_fsave_all_real_baseline_avg)/stc_fsave_all_real_baseline_avg
stc_diff = prepareInverseSolution_group(stc_fsave_all_real_diff_avg, subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)
print('Max val: ', np.max(stc_diff.data))

### Plot single subject sources

In [ ]:
v_ma = 1100
v_mi = 500

showResult('fsaverage', sourceFolder, stc_diff, 
           'real_change_allTogether_norm2z_tp10'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=v_mi, maximum=v_ma)#, med = (v_mi + v_ma)/2)

#Get labels of parcellation
src_ave = mne.read_source_spaces(subjects_dir+'fsaverage\\bem\\fsaverage-ico-5-src.fif')
fsave_vertices = [s['vertno'] for s in src_ave]
label = mne.read_labels_from_annot('fsaverage', parc='aparc',hemi = 'both',
                                   subjects_dir=subjects_dir)
print('All labels: \n', label)

# Initialize a list of labels:
'''
labels_names = stc_diff.data.shape[0]*[None]
labels_inSourceEst = stc_diff.data.shape[0]*[None]

for l_i in range(len(label)):
    #l_i = 0 # label index we want
    # the name for current labels:
    for ver in label[l_i].restrict(src_ave).vertices:
        labels_names[ver] = label[l_i].restrict(src_ave).name
        labels_inSourceEst[ver] = label[l_i].restrict(src_ave)

print(len(stc_diff.data))
print(len(labels_names))
'''

desired_thres = 1000

sign_i = np.where(stc_fsave_all_real_diff_avg > desired_thres)[0]
labels_names_sign = []
for s in sign_i:
    print(labels_names[s])
    labels_names_sign.append(labels_names[s])

print(labels_names_sign)


'''
Brain = mne.viz.get_brain_class()

subjects_dir = mne.datasets.sample.data_path() + '/subjects'
#mne.datasets.fetch_hcp_mmp_parcellation(subjects_dir=subjects_dir,verbose=True)

#mne.datasets.fetch_aparc_sub_parcellation(subjects_dir=subjects_dir,verbose=True)
hemi = 'lh'
brain = Brain('fsaverage', hemi, 'inflated', subjects_dir=subjects_dir,
              cortex='low_contrast', background='white', size=(800, 600))
#brain.add_annotation('HCPMMP1')
labels_to_plot = [l for l in label if l.restrict(src_ave).name in labels_names_sign and hemi in l.name]
for l in labels_to_plot:
    brain.add_label(l, borders=True)

brain.show_view('lateral');
#Image(filename = sourceFolder + 'corticalSources.png', width=600)
brain.save_image(sourceFolder + 'corticalSources_'+hemi+'.png', 'rgb')
'''

# Get MNI coordinates of the peak on left hemisphere
hemi = 'lh'
coordinate_lh = getMNIcoordinates(hemi, stc_diff, subjects_dir)
print('coordinates in left hemi: ', coordinate_lh)

# Get MNI coordinates of the peak on right hemisphere
hemi = 'rh'
coordinate_rh = getMNIcoordinates(hemi, stc_diff, subjects_dir)
print('coordinates in right hemi: ', coordinate_rh)

# Get coordinates of the cluster peak on temporal lobe in right hemisphere
hemi = 'rh'
# find the labels of desired parcellations (e.g. temporal ones) 
labels_rh_temporal = [l for l in label if l.name in labels_names_sign and hemi in l.name][5]
print('labels_rh_temporal: \n', labels_rh_temporal)
coordinate_temporal = getMNIcoordinates(hemi, stc_diff, subjects_dir, labels=labels_rh_temporal)
print('coordinates: ', coordinate_temporal)